In [1]:
from blazingsql import BlazingContext
# start up BlazingSQL
bc = BlazingContext()

BlazingContext ready


### Runtime info & cuDF results
- query `bsql_logs` for runtime info
- have some fun with cuDF methods 
  - note: BlazingSQL query results return as cuDF DataFrames, and can therefore easily make use of cuDF methods

In [2]:
# determine data load time & total time for all queries, showing the latest ones first
log_query = """
            SELECT
                query_id, 
                MAX(load_time) AS load_time, 
                MAX(end_time) as end_time, 
                MAX(total_time) AS total_time 
            FROM (
                SELECT 
                    query_id, 
                    MAX(log_time) AS end_time,
                    SUM(CASE WHEN info = 'evaluate_split_query load_data' THEN duration ELSE 0 END) AS load_time,
                    SUM(CASE WHEN info = 'Query Execution Done' THEN duration ELSE 0 END) AS total_time
                FROM 
                    bsql_logs
                WHERE 
                    info = 'evaluate_split_query load_data' 
                    OR info = 'Query Execution Done'
                GROUP BY 
                    query_id 
                )
            GROUP BY
                query_id
            ORDER BY
                end_time DESC
                """

# save results to parquet w/ cuDF .to_csv() 
bc.log(log_query).to_csv('bsql_log.csv', index=False)

# share discription of results
print(bc.log(log_query).describe())

# let's see a sample w/ .sample() via cuDF .to_pandas()
bc.log(log_query).to_pandas().sample(5)

           query_id  load_time  total_time
count     56.000000  56.000000   56.000000
mean   31109.267857   3.816798    5.130908
std    16458.234992   1.555298    3.305104
min      352.000000   1.191662    2.291305
25%    16223.750000   2.887372    2.989326
50%    31480.000000   3.268019    3.726566
75%    46414.000000   4.259069    6.511285
max    57977.000000   7.628008   15.662748


,query_id,load_time,end_time,total_time
3,17528,5.728122,2020-01-15 00:38:00,10.382488
40,22684,2.648391,2020-01-10 01:28:05,2.758080
53,51427,2.771451,2020-01-10 01:11:20,2.861178
0,31060,7.207241,2020-01-15 00:41:16,12.485735
37,16145,4.607126,2020-01-10 01:34:15,4.728696
